<a href="https://colab.research.google.com/github/pfmoro/VisualizacaoRefugiadosVenezuela/blob/main/Prototype_Oil_x_Refugees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from google.colab import files
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.express as px
import os
from scipy.stats import spearmanr, linregress
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller

In [2]:
# Remove todos os arquivos CSV antigos
# Lista os arquivos atuais no ambiente de trabalho do Colab
files_in_colab = os.listdir('/content')

for file_name in files_in_colab:
    if file_name.endswith('.csv'):
        os.remove(file_name)
        print(f'Arquivo {file_name} removido.')

In [4]:
uploaded = files.upload()

Saving Oil Refugees GDP.csv to Oil Refugees GDP.csv


In [5]:
file_name = 'Oil Refugees GDP.csv'

# Verifica se o arquivo foi carregado corretamente
if file_name in uploaded:
    # Lê o arquivo CSV usando a biblioteca Pandas
    df_oil_ref_gdp = pd.read_csv(file_name,sep=";",encoding='latin-1')
else:
    print(f"O arquivo '{file_name}' não foi encontrado.")

In [6]:
df_oil_ref_gdp = df_oil_ref_gdp.replace('#DIV/0!', pd.NaT if 'datetime' in df_oil_ref_gdp.columns else pd.NA)

In [7]:
df_oil_ref_gdp.head(5)

,Year,Refugees Venezuela,Refugees Venezuela Variation,refugees normalized,Refugees Venezuela Variation %,Crude Oil Year Average,Crude Oil Year Average Variation %,Crude Oil Year Average Variation % Lag +,Crude Oil Year Average Variation % Lag 2+,Crude Oil Year Average Variation % Lag 3+,Crude Oil Year Average Variation % Lag -,GDP,GDP Variation
0,2001.0,798.0,NaN,NaN,NaN,24.351825,-0.137,NaN,NaN,NaN,0.0237,300198.0,NaN
1,2002.0,1304.0,506.0,0.000168,0.63,24.927748,0.024,-0.1374,NaN,NaN,0.1593,277865.0,-0.0744
2,2003.0,2614.0,1310.0,0.000435,1.00,28.898903,0.159,0.0237,-0.1374,NaN,0.3057,261376.0,-0.0593
3,2004.0,3933.0,1319.0,0.000438,0.50,37.733388,0.306,0.1593,0.0237,-0.1374,0.4150,317486.0,0.2147
4,2005.0,5397.0,1464.0,0.000486,0.37,53.391025,0.415,0.3057,0.1593,0.0237,0.2041,361227.0,0.1378


In [8]:
df_oil_ref_gdp.columns

Index(['Year', 'Refugees Venezuela', 'Refugees Venezuela Variation ',
       'refugees normalized', 'Refugees Venezuela Variation %',
       'Crude Oil Year Average', 'Crude Oil Year Average Variation %',
       'Crude Oil Year Average Variation % Lag +',
       'Crude Oil Year Average Variation % Lag 2+',
       'Crude Oil Year Average Variation % Lag 3+',
       'Crude Oil Year Average Variation % Lag -', 'GDP', 'GDP Variation'],
      dtype='object')

Gráfico simples - Refugiados(Normalizado) x Ano

In [12]:
fig = go.Figure([go.Scatter(x=df_oil_ref_gdp['Year'], y=df_oil_ref_gdp['refugees normalized'])])
fig.show()

Teste de estacionariedade - necessário para aplicar causalidade de granger

In [10]:
adfuller(df_oil_ref_gdp['refugees normalized'].dropna(how='any'))

(-3.9032703213890576,
 0.0020109107316031967,
 7,
 15,
 {'1%': -3.9644434814814815,
  '5%': -3.0849081481481484,
  '10%': -2.6818144444444445},
 -8.390805819779033)

In [11]:
adfuller(df_oil_ref_gdp['Crude Oil Year Average Variation %'].dropna(how='any'))

(-4.17700219928775,
 0.0007205178654771845,
 1,
 21,
 {'1%': -3.7883858816542486,
  '5%': -3.013097747543462,
  '10%': -2.6463967573696143},
 6.343296133155931)

Variação do petróleo e refugiados normalizados são estacionários - premissa dos testes de granger são respeitadas

In [15]:
#Teste de causalidade de granger
#Remove NaN para plot e causalidade:
df_plot=df_oil_ref_gdp[['Year','refugees normalized','Crude Oil Year Average Variation %']].dropna(how='any')

gc_res = grangercausalitytests(df_plot[['refugees normalized','Crude Oil Year Average Variation %']],maxlag=6)
gc_res


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4662  , p=0.2416  , df_denom=18, df_num=1
ssr based chi2 test:   chi2=1.7106  , p=0.1909  , df=1
likelihood ratio test: chi2=1.6445  , p=0.1997  , df=1
parameter F test:         F=1.4662  , p=0.2416  , df_denom=18, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=6.0882  , p=0.0116  , df_denom=15, df_num=2
ssr based chi2 test:   chi2=16.2352 , p=0.0003  , df=2
likelihood ratio test: chi2=11.8860 , p=0.0026  , df=2
parameter F test:         F=6.0882  , p=0.0116  , df_denom=15, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=4.4109  , p=0.0261  , df_denom=12, df_num=3
ssr based chi2 test:   chi2=20.9520 , p=0.0001  , df=3
likelihood ratio test: chi2=14.1216 , p=0.0027  , df=3
parameter F test:         F=4.4109  , p=0.0261  , df_denom=12, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.0790  , p=0.0743  , df_d

{1: ({'ssr_ftest': (1.466215855285146, 0.24160317399311723, 18.0, 1),
   'ssr_chi2test': (1.7105851644993373, 0.1909094660004107, 1),
   'lrtest': (1.6444823825396426, 0.19971157514393348, 1),
   'params_ftest': (1.4662158552851705, 0.24160317399311357, 18.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (6.088191848790719, 0.011594364535486382, 15.0, 2),
   'ssr_chi2test': (16.235178263441917, 0.00029824682820723847, 2),
   'lrtest': (11.8859629728612, 0.002624193977105926, 2),
   'params_ftest': (6.088191848790719, 0.011594364535486382, 15.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (4.4109472739816535, 0.026081556703424123, 12.0, 3),
   'ssr_chi2test': (20.951999551412857, 0.00010772043177772099, 3),
   'lrtest': (14.121555119713662, 0.0027442787472139727, 3),
   'params_ftest': (4.410947273981654, 0.02608155670342411, 12.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0

In [27]:
#Construir Dataframe para armazenar os lags do ssr_chi2test (o mais robusto a violações de premissas)
# Inicializa uma lista para armazenar os p-values
p_values_list = []

# Itera sobre os resultados de cada lag e extrai o p-value do ssr_chi2test
for lag, results in gc_res.items():
    p_value = results[0]['ssr_chi2test'][1]
    p_values_list.append((lag, p_value))

# Criando o DataFrame
df_p_values = pd.DataFrame(p_values_list, columns=['lag', 'p_value'])
df_p_values.head(5)

,lag,p_value
0,1,0.190909
1,2,0.000298
2,3,0.000108
3,4,0.000060
4,5,0.000438


In [28]:
def rolling_spearman_slow(seqa, seqb, window):
    stridea = seqa.strides[0]
    ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
    strideb = seqa.strides[0]
    ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
    corrs = [spearmanr(a, b)[0] for (a,b) in zip(ssa, ssb)]
    return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)


In [41]:
#Remove NaN para plot:
var_y_opt=['Crude Oil Year Average Variation %',
       'Crude Oil Year Average Variation % Lag +',
       'Crude Oil Year Average Variation % Lag 2+',
       'Crude Oil Year Average Variation % Lag 3+']

var_x='refugees normalized'
var_y=var_y_opt[0]
window=5

df_plot=df_oil_ref_gdp[['Year', var_x, var_y]].dropna(how='any')
df_plot[var_x]=df_plot[var_x].astype(float)
df_plot[var_y]=df_plot[var_y].astype(float)
#Criar correlação Rolante:
df_plot['correlacao_rolante']=rolling_spearman_slow(df_plot[var_x].astype(float).values,df_plot[var_y].astype(float).values,window=window)
#residuos de modelo linear ajustado:
slope, intercept, r_value, p_value, std_err = linregress(df_plot[var_x], df_plot[var_y])
df_plot['y_pred'] = slope * df_plot[var_x] + intercept
df_plot['residuos']=df_plot[var_y]-df_plot['y_pred']

#Usaremos subplots:
fig = make_subplots(rows=3, cols=2)

# Cria um gráfico de dispersão para var_x
trace1 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot[var_x].astype(float),
    mode='lines+markers',
    name=var_x
)

# Cria um gráfico de dispersão para var_y
trace2 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot[var_y].astype(float),
    mode='lines+markers',
    name=var_y
)

# Gráfico de correlação (usaremos um scatter plot simples para demonstrar)
trace3 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot['correlacao_rolante'],
    mode='lines',
    name='Correlação Rolante (Spearman)'
)

# Gráfico de resíduos
trace4 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot['residuos'],
    mode='lines+markers',
    name='Resíduos'
)

# Gráfico de pvalue dos lags Granger
trace5 = go.Bar(
    x=df_p_values['lag'],
    y=1-df_p_values['p_value'],
    name='Significância do Atraso (causalidade Granger)'
)

layout_trace5 = go.Layout(
    xaxis=dict(title='Lag', dtick=1),  # dtick=1 para mostrar todos os lags no eixo x
    yaxis=dict(title='1 - p-value', range=[0, 1], tickformat='.2f'),  # Formata os ticks para 2 casas decimais
    shapes=[
        dict(
            type='line',
            x0=0,
            y0=0.95,
            x1=df_p_values['lag'].max(),
            y1=0.95,
            line=dict(
                color='red',
                width=2,
                dash='dashdot',  # Pode ser 'solid', 'dash', 'dot' ou 'dashdot'
            ),
            layer='below'  # Garante que a linha esteja atrás das barras
        )
    ],
    title='Significância dos Lags na Causalidade de Granger',
    showlegend=False  # Pode ser True se você quiser mostrar a legenda apenas para este subplot
)


# Encontrar os índices onde a correlação em módulo é maior que 0.7
indices_correlacao_alta = np.where(np.abs(df_plot['correlacao_rolante']) > 0.7)[0]

# Inicializar uma lista para armazenar os intervalos de início e fim
intervalos = []
# Encontrar os subintervalos consecutivos
for i in range(len(indices_correlacao_alta)):
    if i == 0 or indices_correlacao_alta[i] != indices_correlacao_alta[i-1] + 1:
        # Início de um novo intervalo
        inicio = indices_correlacao_alta[i]
    if i == len(indices_correlacao_alta) - 1 or indices_correlacao_alta[i] != indices_correlacao_alta[i+1] - 1:
        # Fim de um intervalo
        fim = indices_correlacao_alta[i]
        intervalos.append((inicio, fim))

# Define os dados que serão plotados
data = [trace1, trace2, trace3, trace4, trace5]
fig.add_trace(trace1,row=1, col=1)
fig.add_trace(trace2,row=1, col=1)
fig.add_trace(trace3,row=2, col=1)
fig.add_trace(trace4,row=3, col=1)
fig.add_trace(trace5,row=1, col=2)

# Adicionando a linha horizontal apenas para o trace5 (segundo subplot)
fig.add_shape(
    dict(
        x0=0,
        y0=0.95,
        x1=df_p_values['lag'].max(),
        y1=0.95,
        line=dict(
            color='red',
            width=2,
            dash='dashdot',
        ),
    ),
    row=1,
    col=2,
)

fig.update_layout(height=600, width=800,
                  title_text="Comparação entre {} e {}, mais Correlação e Resíduos".format(var_x, var_y),
                  shapes=[
        # Adicionar uma forma para cada intervalo de alta correlação
        dict(
            type='rect',
            x0=df_plot['Year'].iloc[inicio],
            y0=-1, # Aqui você pode querer ajustar para o mínimo da sua nova variável var_x
            x1=df_plot['Year'].iloc[fim],
            y1=1, # Aqui você pode querer ajustar para o máximo da sua nova variável var_x
            fillcolor='Lightgrey',
            opacity=0.5,
            line=dict(
                width=0,
            ),
        )
        for inicio, fim in intervalos
    ]
                  )



# Plota o gráfico no Google Colab
#plot(fig)
fig.show()
